In [655]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta, datetime

'''
Definitions
'''

#input file name
DATA_FILE = "fuel_levels50.csv"

#original columns names
FUEL_LEVEL = "fuel_level"
TIME = "time"
VEHICLE_ID = "vehicle_id"
ENGINE_ON = "engine_on"
PROVIDER_ID = "provider_id"

#new columns names
#columns starting with "roll" are caculted on rolling window
DATETIME = 'datetime'
DELTA_TIME = "delta_time"
DELTA_FUEL = "delta_fuel"
IS_THEFT = "suspicion_of_theft"

ROLL_TIME = "rolling_time"
ROLL_FUEL = "rolling_fuel"
ROLL_RATE = "rolling_rate"

#constants
QUNTILE =  0.2 #quntile
MIN_AMOUNT = 20 # minimum amount of fuel to consider as a theft (liter)

#display option
pd.options.display.float_format = '{:.3f}'.format


In [656]:
'''
Read Data
'''

#read data into a data fraae and drop index column 
df = pd.read_csv(DATA_FILE)
df.drop(df.columns[0],axis=1,inplace=True)

#show head
df.head()

,engine_on,time,fuel_level,provider_id,vehicle_id
0,1,1448738636105.800,178.836,2,26
1,1,1449367017443.487,23.550,1,21
2,1,1445733458105.648,27.928,2,40
3,1,1445192368945.336,29.950,1,22
4,1,1448686185252.171,253.891,2,12


In [ ]:
'''
Data Preparation
'''

#sort by id and time, and reset index 
df.sort_values(by=[VEHICLE_ID, TIME], inplace=True)
df = df.reset_index(drop=True)

#group by vehicle id
groups = df.groupby(VEHICLE_ID, sort=False)

#smooth data of fuel level with moving average method
df[FUEL_LEVEL] = groups[FUEL_LEVEL].apply(lambda g: g.rolling(3, min_periods=1).mean())

#calculate delta time and delta fuel level
df[DELTA_TIME] = groups[TIME].diff()
df[DELTA_FUEL] = groups[FUEL_LEVEL].diff()

#add datetime column
df[DATETIME] = pd.to_datetime(df[TIME],unit='ms')

In [ ]:
df.head()

In [ ]:
# fisrt - get a little bit more familiar with the data (prints only)

#total number of observations  
print("Total num observations: {}".format(df.shape[0]))

#num observations when engine on / off
num_engine_on = df[df[ENGINE_ON]==1].shape[0]
num_engine_off = df[df[ENGINE_ON]==0].shape[0]
print("Num observations when engine on: {}".format(num_engine_on))
print("Num observations when engine off: {}".format(num_engine_off))

#show total time
print("\nDuration")
print (df.groupby([VEHICLE_ID])
       .apply(lambda x: x[DATETIME].iloc[-1]-x[DATETIME].iloc[0]))

#print some statistics on data
print("\nStatistics for fuel-level:")
print (df.groupby(VEHICLE_ID)[FUEL_LEVEL].describe())

print("\nStatistics for delta_time:")
print (df.groupby(VEHICLE_ID)[DELTA_TIME].describe())

print("\nStatistics for delta_fuel:")
print (df.groupby(VEHICLE_ID)[DELTA_FUEL].describe())

In [ ]:
'''
calculate fuel-level change rate 
'''

def minutes_to_window_size(minutes,interval): 
    '''
    define a window size for the rolling window operations 
    gets delta time in minutes and intrval, and find
    what size of a window (i.e., num observations) 
    captures (at least) this delta time 
    '''
    milliseconds = minutes*60*1000
    window_size = milliseconds / interval
    return int(window_size)

def rolling_sum(df, window, min_periods):
    return df.rolling(window, min_periods=min_periods).sum()

#define a window size 
interval = df[DELTA_TIME].median()
win = minutes_to_window_size(10, interval)
min_per = minutes_to_window_size(1, interval)

# calculte difference rate in fuel-level.
# change rate in fuel-level is the change amount in fuel level 
# divided by change amount in time, along a specified window. 
# units: liters/millisecond 
df[ROLL_FUEL] = groups[DELTA_FUEL].apply(rolling_sum,  win, min_per)
df[ROLL_TIME] = groups[DELTA_TIME].apply(rolling_sum, win, min_per)                                                  
df[ROLL_RATE] = df[ROLL_FUEL] / df[ROLL_TIME]

#print(df[[ROLL_FUEL, ROLL_TIME, ROLL_RATE]])

In [ ]:
'''
Find possible thefts
'''

def find_thefts_on(df, min_stolen_amount, quntile):
    '''
    Find possible thefts when engine on
    '''
    
    #get observations where there is a negative difference in fuel level
    #and caculate the specified quntile
    thershold = df.loc[df[ROLL_RATE] < 0, ROLL_RATE].quantile(quntile)
   
    #find sharp decreases in fuel level
    #by getting rates values under the specified quntile 
    #(i.e. large absolute values of rates)
    df[IS_THEFT] = (df[ROLL_RATE] < 0) \
                    & (df[ROLL_RATE] < thershold) \
                    & (df[ROLL_FUEL].abs() > min_stolen_amount) 
    
    return df[[IS_THEFT]]


def find_thefts_off(df, min_stolen_amount, minutes=3):
    '''
    Find possible thefts when engine off
    '''
   
    df['index'] = df.index 
    df.set_index([DATETIME], inplace=True)
    
    #for each observation when engine is off 
    #and is the first observation after switching on
    datetimes = df[(df[ENGINE_ON]==0) & (df[DELTA_TIME]>2000)].index
    true_indexes = []
    for datetime in datetimes:
        
        #get average fuel-level a few minutes before powering off
        t = int(df.loc[datetime,DELTA_TIME])
        start_datetime = datetime - (pd.offsets.Milli(t) + pd.offsets.Minute(minutes))
        before = df[start_datetime : datetime][FUEL_LEVEL].mean()
        
        #get average fuel-level a few minutes after turning on
        end_datetime = datetime + pd.offsets.Minute(minutes)
        after = df[datetime : end_datetime][FUEL_LEVEL].mean()
        
        #if difference before powering off is greater than 
        #the specified minimal amount of stolen fuel-level, save index
        if before - after > min_stolen_amount:
            print(before, after )
            true_indexes.append(datetime)
    
    df[IS_THEFT] = False
    df.loc[true_indexes, IS_THEFT] = True 
    
    return df.set_index(['index'])[[IS_THEFT]]
  
#find observations which are possibly theft cases
thefts_on = groups.apply(find_thefts_on, MIN_AMOUNT, QUNTILE)[IS_THEFT]
thefts_off = groups.apply(find_thefts_off, MIN_AMOUNT)[IS_THEFT]

#combine to one column
df[IS_THEFT] = thefts_on | thefts_off


In [ ]:
'''
save results to file
'''

#extract relevant columns and save to file 
result = df.loc[df[IS_THEFT], [VEHICLE_ID, TIME]]
result.reset_index(inplace=True, drop=True)

#print(result)

result.to_csv("fuel_thefts.csv")


In [ ]:
'''
Plot regions of thefts
'''

def plot_thefts(df,title):
    
    plt.figure(figsize=(30,20))
    plt.title(title, fontsize=20)
    
    #plot fuel-level along datetime
    time_serial = df[FUEL_LEVEL]
    time_serial.plot(fontsize=18)
    
    #plot in green regions of engine 
    time_serial3 = df.loc[df[ENGINE_ON] == 0, FUEL_LEVEL]
    time_serial3.plot(style="s", ms=10, c='green')
    
    #plot in green regions of off thefts 
    time_serial2 = df.loc[df[IS_THEFT] == True, FUEL_LEVEL]
    time_serial2.plot(style=".", ms=10, c='orange')
    
    
for name, group in groups:
    
    vehicle = groups.get_group(0).reset_index()
    vehicle.set_index(DATETIME, inplace=True)
    
    #plot vehicle from start to end
    time_range = vehicle.index[-1] - vehicle.index[0]
    title = "vehicle_id = {}, time range: {}".format(name, time_range)
    plot_thefts(vehicle, title)
    
    #zoom-in to a specified region (manual =( )
    start = datetime(2015, 11, 10, 12)
    end = datetime(2015, 11, 11, 0)
    vehicle_zooom_in = vehicle[start:end]
    
    time_range = end - start
    title = "vehicle_id = {}, time range: {}".format(name, time_range)
    plot_thefts(vehicle_zooom_in, title)
    
    
    break
    


In [ ]:
'''
thefts statistics
'''

TIME_INTERVAL = '30T'


def sum_stolen_amount(df):
    
    #if there not a theft case in given time interval
    #return 0
    if df[IS_THEFT].isnull().all():
        return 0
    
    #otherwise return sum of delta fuel in that time interval
    if df[IS_THEFT].any():
        return df[DELTA_FUEL].sum()   

groups = df.groupby([VEHICLE_ID])

stolen_amounts = []
for name, group in groups:
    
    #resample by the specified time and calcualte stolen amounts
    group = group.set_index(DATETIME) 
    amounts = group.resample(TIME_INTERVAL).apply(sum_stolen_amount)
    
    #add stolen amounts of current group
    stolen_amounts.extend(amounts[amounts < 0].abs())
    

print ("Avg stolen fuel amount: {} liter".format(np.mean(amounts_stolen)))